#### What is KNN?
A database with several data points that are separated into different classes. KNN is used by such databases to predict the classification of the new data point. Everything hinges on the distance. So the class of a given data point is predicted based on how close it is to the samples.

#### Data Dictionary
1. Number of Instances: 699 
2. Number of Attributes: 10 plus the class attribute
3. Attribute Information: (class attribute has been moved to last column)

      Attribute,                Range
   1. Sample code number            ,id number
   2. Clump Thickness               ,1 - 10
   3. Uniformity of Cell Size       ,1 - 10
   4. Uniformity of Cell Shape      ,1 - 10
   5. Marginal Adhesion             ,1 - 10
   6. Single Epithelial Cell Size   ,1 - 10
   7. Bare Nuclei                   ,1 - 10
   8. Bland Chromatin               ,1 - 10
   9. Normal Nucleoli               ,1 - 10
  10. Mitoses                       ,1 - 10
  11. Class:                        ,(2 for benign, 4 for malignant)

4. Missing attribute values: 16

   There are 16 instances in Groups 1 to 6 that contain a single missing 
   (i.e., unavailable) attribute value, now denoted by "?".  

5. Class distribution:
 
   Benign: 458 (65.5%)
   Malignant: 241 (34.5%)


In [51]:
#make the necessary imports
import numpy as np
from sklearn import preprocessing, neighbors
from sklearn.model_selection import train_test_split
import pandas as pd

In [52]:
#import the dataset
df = pd.read_csv('breast-cancer-wisconsin.csv')

In [53]:
df.head()

,id,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [54]:
#We know from the metadata that there are missing values denoted as '?'. Replace them with -99999 since most algorithms 
#treat it as an outlier
df.replace('?', -99999, inplace = True)
#drop the id column since it has no implication on the dataset
df.drop(['id'], 1, inplace = True)
#some values are treated as text hence we convert the dtype of all values to be float
full_data = df.astype(float).values.tolist()

In [55]:
#remove unnecessary space in the column names
df.columns = df.columns.str.replace(' ', '')

In [56]:
#split the data into X and y. X has all the features and y has the label data array.
X = np.array(df.drop(['class'], 1))
y = np.array(df['class'])

In [57]:
#split the data into train and test set. Here we have set to use 20% of the dataset for testing purposes.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [58]:
#we fit the train dataset to the KNN classifier 
clf = neighbors.KNeighborsClassifier()
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [107]:
#97% accuracy for the given dataset is not bad. So this means the beningn and malignant classifications in the dataset
#is about 97% accurate
accuracy = clf.score(X_test, y_test)
accuracy

0.9571428571428572

In [60]:
#Lets use this to make a prediction
example_measures = np.array([[4, 2, 1, 1, 1, 2, 3, 2, 1], [4, 2, 1, 1, 1, 2, 3, 2, 1]])
example_measures = example_measures.reshape(len(example_measures), -1)
prediction = clf.predict(example_measures)
prediction

array([2, 2])

#### Building our own KNN classifier algorithm

In [63]:
import numpy as np
from math import sqrt
import warnings
from collections import Counter
import pandas as pd
import random

In [103]:
def k_nearest_neighbors(data, predict, k = 3):
    if len(data) >= k:
        warnings.warn('k is set to a value less than total voting groups')
    distances = []
    for group in data:
        for features in data[group]:
            euclidean_distance = np.linalg.norm(np.array(features)- np.array(predict))
            distances.append([euclidean_distance, group])
    
    votes = [i[1] for i in sorted(distances)[:k]]
    
    vote_result = Counter(votes).most_common(1)[0][0]
    
    return vote_result

In [95]:
#shuffle the full_data before splitting it as train and test
random.shuffle(full_data)

In [96]:
#alot 20% of the data to test
test_size = 0.2
#create a list of list values for the given labels
train_set = {2: [], 4: []}
test_set = {2: [], 4: []}

#80% of the data for training
train_data = full_data[:-int(len(full_data)*test_size)]
#20% of the data for testing
test_data = full_data[-int(len(full_data)*test_size):]

In [97]:
#Take all the values until the class label and send it to the train and test data sets
for i in train_data:
    train_set[i[-1]].append(i[:-1])
    
for i in test_data:
    test_set[i[-1]].append(i[:-1])

In [106]:
correct = 0
total   = 0

#for every 2 or 4 groups in test_set
for group in test_set:
    #for every list in the group
    for data in test_set[group]:
        #pass the train_set, full_data and the default k value
        vote = k_nearest_neighbors(train_set, data, k= 5)
        if vote == group:
            correct += 1
        total += 1
        
print('Accuracy:', correct/total)

Accuracy: 0.9784172661870504


The accuracy of this algorithm is pretty close to that of the KNN sklearn algorithm.